# Predict a country's main religion based mostly on its flag's details
## Using World Flags Data

<img src='flags.JPG'>

A critical task for data analysis is often aggregating or transforming groups of data. After preparing your data, you may need to compute group statistics or possible pivot tables for reporting or visualization purposes. Pandas `groupby` is a flexible way to perform these aggregations and summarize datasets.

For this module, we will be working with data that contains details of various nations and their flags. It was originally collected from the 'Collins Gen Guide to Flags' from Collins Publishers in 1986. Note that this data is out-of-date. For instance, it still includes 'USSR' as a country.
      
Here is some basic information about the dataset:

- There are 194 instances (aka rows).
- There are 30 attributes in total (aka columns).
- 10 attributes are numeric-valued.  The remainder are either Boolean or nominal-valued.
- There are no missing values.

**Attribute Information**

1. name: Name of the country concerned
2. landmass: 1=N.America, 2=S.America, 3=Europe, 4=Africa, 5=Asia, 6=Oceania
3. zone: Geographic quadrant, based on Greenwich and the Equator (1=NE, 2=SE, 3=SW, 4=NW)
4. area: in thousands of square km
5. population: in round millions
6. language: 1=English, 2=Spanish, 3=French, 4=German, 5=Slavic, 6=Other Indo-European, 7=Chinese, 8=Arabic,            9=Japanese/Turkish/Finnish/Magyar, 10=Others
7. religion: 0=Catholic, 1=Other Christian, 2=Muslim, 3=Buddhist, 4=Hindu, 5=Ethnic, 6=Marxist, 7=Others
8. bars: Number of vertical bars in the flag
9. stripes: Number of horizontal stripes in the flag
10. colors: Number of different colors in the flag
11. red: 0 if red absent, 1 if red present in the flag
12. green: same for green
13. blue: same for blue
14. gold: same for gold (also yellow)
15. white: same for white
16. black: same for black
17. orange: same for orange (also brown)
18. mainhue: predominant colour in the flag (tie-breaks decided by taking the topmost hue, if that fails then the most central hue, and if that fails the leftmost hue)
19. circles: Number of circles in the flag
20. crosses: Number of (upright) crosses
21. saltires: Number of diagonal crosses
22. quarters: Number of quartered sections
23. sunstars: Number of sun or star symbols
24. crescent: 1 if a crescent moon symbol present, else 0
25. triangle: 1 if any triangles present, 0 otherwise
26. icon: 1 if an inanimate image present (e.g., a boat), otherwise 0
27. animate: 1 if an animate image (e.g., an eagle, a tree, a human hand) present, 0 otherwise
28. text: 1 if any letters or writing on the flag (e.g., a motto or slogan), 0 otherwise
29. topleft: color in the top-left corner (moving right to decide tie-breaks)
30. botright: color in the bottom-left corner (moving left to decide tie-breaks)

## Initial Imports

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 50)

## Initial Exploration of Flags Dataset

In [2]:
# create list of column names
columns = ['name','landmass','zone','area','population','language','religion','num_bars','num_stripes','num_colors',
           'red','green','blue','gold','white','black','orange','mainhue','num_circles','num_crosses','num_saltires',
           'num_quarters','num_sunstars','crescent','triangle','icon','animate','text','topleft_color','botright_color']

# import data and show first five rows
flags = pd.read_csv('flag.data', names=columns)
flags.head()

,name,landmass,zone,area,population,language,religion,num_bars,num_stripes,num_colors,red,green,blue,gold,white,black,orange,mainhue,num_circles,num_crosses,num_saltires,num_quarters,num_sunstars,crescent,triangle,icon,animate,text,topleft_color,botright_color
0,Afghanistan,5,1,648,16,10,2,0,3,5,1,1,0,1,1,1,0,green,0,0,0,0,1,0,0,1,0,0,black,green
1,Albania,3,1,29,3,6,6,0,0,3,1,0,0,1,0,1,0,red,0,0,0,0,1,0,0,0,1,0,red,red
2,Algeria,4,1,2388,20,8,2,2,0,3,1,1,0,0,1,0,0,green,0,0,0,0,1,1,0,0,0,0,green,white
3,American-Samoa,6,3,0,0,1,1,0,0,5,1,0,1,1,1,0,1,blue,0,0,0,0,0,0,1,1,1,0,blue,red
4,Andorra,3,1,0,0,6,0,3,0,3,1,0,1,1,0,0,0,gold,0,0,0,0,0,0,0,0,0,0,blue,red


In [3]:
# check size of dataset
flags.shape

(194, 30)

In [4]:
# check general information about dataset
flags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194 entries, 0 to 193
Data columns (total 30 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   name            194 non-null    object
 1   landmass        194 non-null    int64 
 2   zone            194 non-null    int64 
 3   area            194 non-null    int64 
 4   population      194 non-null    int64 
 5   language        194 non-null    int64 
 6   religion        194 non-null    int64 
 7   num_bars        194 non-null    int64 
 8   num_stripes     194 non-null    int64 
 9   num_colors      194 non-null    int64 
 10  red             194 non-null    int64 
 11  green           194 non-null    int64 
 12  blue            194 non-null    int64 
 13  gold            194 non-null    int64 
 14  white           194 non-null    int64 
 15  black           194 non-null    int64 
 16  orange          194 non-null    int64 
 17  mainhue         194 non-null    object
 18  num_circle

In [5]:
# check general statistical information
flags.describe()

,landmass,zone,area,population,language,religion,num_bars,num_stripes,num_colors,red,green,blue,gold,white,black,orange,num_circles,num_crosses,num_saltires,num_quarters,num_sunstars,crescent,triangle,icon,animate,text
count,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000
mean,3.572165,2.211340,700.046392,23.268041,5.340206,2.190722,0.453608,1.551546,3.463918,0.788660,0.469072,0.510309,0.469072,0.752577,0.268041,0.134021,0.170103,0.149485,0.092784,0.149485,1.386598,0.056701,0.139175,0.252577,0.201031,0.082474
std,1.553018,1.308274,2170.927932,91.934085,3.496517,2.061167,1.038339,2.328005,1.300154,0.409315,0.500334,0.501187,0.500334,0.432631,0.444085,0.341556,0.463075,0.385387,0.290879,0.435860,4.396186,0.231869,0.347025,0.435615,0.401808,0.275798
min,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,1.000000,9.000000,0.000000,2.000000,1.000000,0.000000,0.000000,3.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,4.000000,2.000000,111.000000,4.000000,6.000000,1.000000,0.000000,0.000000,3.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,5.000000,4.000000,471.250000,14.000000,9.000000,4.000000,0.000000,3.000000,4.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.750000,0.000000,0.000000
max,6.000000,4.000000,22402.000000,1008.000000,10.000000,7.000000,5.000000,14.000000,8.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,4.000000,2.000000,1.000000,4.000000,50.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## GroupBy Mechanics
### Basic Grouping

Hadley Wickham, an author of many popular packages for the R programming language, coined the term split-apply-combine for describing group operations. 

- First, the data is split into groups.
- Second, a function is applied to each group
- Finally, the results are combined into a result object

Here is a mockup of a simple group aggregation.

<img src='split_apply_combine.JPG'>

*Hadley Wickham, an author of many popular packages for the R programming language, coined the term split-apply-combine for describing group operations.*

To get started, let's create a small dataset.

In [6]:
# create sample dataset with random data
df = pd.DataFrame({'studio_key' : ['Marvel', 'Marvel', 'DC', 'DC', 'Marvel'],
     'department_key' : ['Production', 'Advertising', 'Production', 'Advertising', 'Production'],
     'data1' : [10,6,2,7,5],
     'data2' : [-1,4,-6,5,11]})
df

,studio_key,department_key,data1,data2
0,Marvel,Production,10,-1
1,Marvel,Advertising,6,4
2,DC,Production,2,-6
3,DC,Advertising,7,5
4,Marvel,Production,5,11


In [128]:
df.loc['data1':'data2']

,studio_key,department_key,data1,data2


Suppose we wanted to compute the mean of `data1` grouped by the `studio_key` column.

In [7]:
# create Series GroupBy object using 'studio_key'
grouped = df['data1'].groupby(df['studio_key'])
grouped

In [131]:
grouped77 = df['data1'].groupby(df['studio_key']).mean()
grouped77

studio_key
DC        4.5
Marvel    7.0
Name: data1, dtype: float64

Now we can simply call the 'mean' method on the GroupBy object

In [8]:
# produces new Series
grouped.mean()

studio_key
DC        4.5
Marvel    7.0
Name: data1, dtype: float64

In [9]:
# chaining it all together
grouped = df['data1'].groupby(df['studio_key']).mean()
grouped

studio_key
DC        4.5
Marvel    7.0
Name: data1, dtype: float64

We can also easily pass multiple keys to be used by the GroupBy object. This actually creates a multi-index Series.

In [10]:
# passing multiple Series as a list
means = df['data1'].groupby([df['studio_key'], df['department_key']]).mean()
means

studio_key  department_key
DC          Advertising       7.0
            Production        2.0
Marvel      Advertising       6.0
            Production        7.5
Name: data1, dtype: float64

In [124]:
means2 = df[['data1']].groupby([df['studio_key'], df['department_key']]).mean()
means2

data1
studio_key department_key       
DC         Advertising       7.0
           Production        2.0
Marvel     Advertising       6.0
           Production        7.5

Remember that you can use `unstack()` to produce a DataFrame

In [11]:
# using unstack() to create a DataFrame
means.unstack()

department_key,Advertising,Production
studio_key,,
DC,7.0,2.0
Marvel,6.0,7.5


Frequently the grouping information is found in the same DataFrame as the data you want to work on. In that case, you can pass column names  as the group keys.

In [12]:
# passing DataFrame column names as group keys
df.groupby('studio_key').mean() # output no department key due to its not numerical data

# df['data1'].groupby('studio_key').mean() #produces error because its groupby the series not groupby df
# df['data1'].groupby(df['studio_key']).mean() #produces Series # if groupby series must use series([])

,data1,data2
studio_key,,
DC,4.5,-0.500000
Marvel,7.0,4.666667


Notice that there is no `department_key` in the above result. Since that column is not numeric, it is excluded from the result. By default, all of the numeric columns are aggregated, though it is possible to filter down to a subset as we will soon see.

A very useful GroupBy method is `size`, which returns a Series containing group sizes.

In [13]:
# show group sizes
df.groupby(['studio_key', 'department_key']).size()# this will return a series containing group sizes

studio_key  department_key
DC          Advertising       1
            Production        1
Marvel      Advertising       1
            Production        2
dtype: int64

Instantiate (create) a SeriesGroupBy object called `grouped_flags` that selects `population` from the `flags` dataset and groups it by `landmass`

*Note: landmass: 1=N.America, 2=S.America, 3=Europe, 4=Africa, 5=Asia, 6=Oceania*

In [14]:
flags.head()

,name,landmass,zone,area,population,language,religion,num_bars,num_stripes,num_colors,red,green,blue,gold,white,black,orange,mainhue,num_circles,num_crosses,num_saltires,num_quarters,num_sunstars,crescent,triangle,icon,animate,text,topleft_color,botright_color
0,Afghanistan,5,1,648,16,10,2,0,3,5,1,1,0,1,1,1,0,green,0,0,0,0,1,0,0,1,0,0,black,green
1,Albania,3,1,29,3,6,6,0,0,3,1,0,0,1,0,1,0,red,0,0,0,0,1,0,0,0,1,0,red,red
2,Algeria,4,1,2388,20,8,2,2,0,3,1,1,0,0,1,0,0,green,0,0,0,0,1,1,0,0,0,0,green,white
3,American-Samoa,6,3,0,0,1,1,0,0,5,1,0,1,1,1,0,1,blue,0,0,0,0,0,0,1,1,1,0,blue,red
4,Andorra,3,1,0,0,6,0,3,0,3,1,0,1,1,0,0,0,gold,0,0,0,0,0,0,0,0,0,0,blue,red


In [15]:
# instantiate a SeriesGroupby object
grouped_flags= flags['population'].groupby(flags['landmass'])
grouped_flags # that create a Series groupby object

Using the `grouped_flags` object, what is the average population by landmass? What is the minimum population by landmass? What is the maximum population by landmass?

In [16]:
# find the average population by landmass
grouped_flags.mean()

landmass
1    12.290323
2    15.705882
3    13.857143
4     8.788462
5    69.179487
6    11.300000
Name: population, dtype: float64

In [17]:
# find the minimum population by landmass
grouped_flags.min()

landmass
1    0
2    0
3    0
4    0
5    0
6    0
Name: population, dtype: int64

In [18]:
# find the maximum population by landmass
grouped_flags.max()

landmass
1     231
2     119
3      61
4      56
5    1008
6     157
Name: population, dtype: int64

Instantiate an object called `flag_means` that selects the `population` and groups it by `zone`, then by `landmass` and calculates the mean of the population.

*Note: zone: Geographic quadrant, based on Greenwich and the Equator (1=NE, 2=SE, 3=SW, 4=NW)*

In [19]:
# population grouped by zone, then by landmass and calculates the mean of the population and called flag_means
flags_means = flags['population'].groupby([flags['zone'], flags['landmass']]).mean()

flags_means

zone  landmass
1     3           13.500000
      4           12.789474
      5           69.179487
      6            9.600000
2     4            7.315789
      6           17.800000
3     2           22.000000
      4            0.000000
      6            0.000000
4     1           12.290323
      2            6.714286
      3           15.285714
      4            5.769231
Name: population, dtype: float64

Turn `flag_means` into a DataFrame with `zone` as the rows and `landmass` as the columns. You should be able to do this using one pandas method. Take note of the missing values in the new DataFrame.

In [20]:
#Turn flag_means into a DataFrame with zone as the rows and landmass as the columns
flags_means.unstack()

landmass,1,2,3,4,5,6
zone,,,,,,
1,NaN,NaN,13.500000,12.789474,69.179487,9.6
2,NaN,NaN,NaN,7.315789,NaN,17.8
3,NaN,22.000000,NaN,0.000000,NaN,0.0
4,12.290323,6.714286,15.285714,5.769231,NaN,NaN


Group the entire `flags` dataset by `landmass` and compute the median of each numeric column.

In [21]:
# Group the entire `flags` dataset by `landmass` and compute the median of each numeric column.
flags.groupby('landmass').median()

,zone,area,population,language,religion,num_bars,num_stripes,num_colors,red,green,blue,gold,white,black,orange,num_circles,num_crosses,num_saltires,num_quarters,num_sunstars,crescent,triangle,icon,animate,text
landmass,,,,,,,,,,,,,,,,,,,,,,,,,
1,4.0,9.0,0.0,1.0,1.0,0.0,0.0,3.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.0,407.0,6.0,2.0,0.0,0.0,3.0,3.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,92.0,8.0,6.0,1.0,0.0,0.0,3.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2.0,298.5,5.0,8.0,5.0,0.0,1.0,3.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.0,185.0,10.0,8.0,2.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
6,2.0,2.0,0.0,1.0,1.0,0.0,0.0,4.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,0.0,0.0


How many countries are represented in each group if you group by `landmass` and then `zone`.

In [22]:
#number of countries, group by landmass and then zone
flags.groupby([flags['landmass'], flags['zone']]).size()#,flags['name']

landmass  zone
1         4       31
2         3       10
          4        7
3         1       28
          4        7
4         1       19
          2       19
          3        1
          4       13
5         1       39
6         1        5
          2       10
          3        5
dtype: int64

### Iterating Over Groups

The GroupBy object supports iteration, generating a sequence of 2-tuples containing the group name along with the chunk of data.

Let's remind ourselves of our sample dataset.

In [23]:
# view DataFrame
df

,studio_key,department_key,data1,data2
0,Marvel,Production,10,-1
1,Marvel,Advertising,6,4
2,DC,Production,2,-6
3,DC,Advertising,7,5
4,Marvel,Production,5,11


In [24]:
# reminder: this creates a DataFrameGroupBy object
df.groupby('studio_key')

You can iterate through this object to find the name and group data.

In [25]:
# loop through object
for name, group in df.groupby('studio_key'):
    print(name)
    print(group)
    print('----')

DC
  studio_key department_key  data1  data2
2         DC     Production      2     -6
3         DC    Advertising      7      5
----
Marvel
  studio_key department_key  data1  data2
0     Marvel     Production     10     -1
1     Marvel    Advertising      6      4
4     Marvel     Production      5     11
----


In the case of multiple keys, the first element in the tuple will be a tuple of key values.

In [26]:
# loop through object
print(group)
for (key1, key2), group in df.groupby(['studio_key', 'department_key']): # (key1 key2)is tuple of the object key value
    print(f'key1: {key1}')
    print(f'key2: {key2}')
    print(group)
    print('----') 

  studio_key department_key  data1  data2
0     Marvel     Production     10     -1
1     Marvel    Advertising      6      4
4     Marvel     Production      5     11
key1: DC
key2: Advertising
  studio_key department_key  data1  data2
3         DC    Advertising      7      5
----
key1: DC
key2: Production
  studio_key department_key  data1  data2
2         DC     Production      2     -6
----
key1: Marvel
key2: Advertising
  studio_key department_key  data1  data2
1     Marvel    Advertising      6      4
----
key1: Marvel
key2: Production
  studio_key department_key  data1  data2
0     Marvel     Production     10     -1
4     Marvel     Production      5     11
----


By default `groupby` groups on axis=0, but you can group on any of the other axes. For example, we could group the columns of our example `df` here by `dtype` like so:

In [27]:
# check data types, # we can group on any axis
df.dtypes

studio_key        object
department_key    object
data1              int64
data2              int64
dtype: object

In [28]:
# print data type and respective group data
grouped = df.groupby(df.dtypes, axis=1)# create an object
for dtype, group in grouped:
    print(dtype)#grouped integer data type and object information
    print(group)# grouped object data and object information
    print('----')

int64
   data1  data2
0     10     -1
1      6      4
2      2     -6
3      7      5
4      5     11
----
object
  studio_key department_key
0     Marvel     Production
1     Marvel    Advertising
2         DC     Production
3         DC    Advertising
4     Marvel     Production
----


### Selecting a Column or Subset of Columns

In [29]:
# groupby studio_key, average of 'data1', returns Series
df['data1'].groupby(df['studio_key']).mean()

studio_key
DC        4.5
Marvel    7.0
Name: data1, dtype: float64

In [30]:
# syntactic sugar for above
df.groupby('studio_key')['data1'].mean()

studio_key
DC        4.5
Marvel    7.0
Name: data1, dtype: float64

In [31]:
# groupby 'studio_key', average of data2, returns DataFrame
df[['data2']].groupby(df['studio_key']).mean()

,data2
studio_key,
DC,-0.500000
Marvel,4.666667


In [32]:
# syntactic sugar for above
df.groupby('studio_key')[['data2']].mean() # this method more easy

,data2
studio_key,
DC,-0.500000
Marvel,4.666667


Especially for large datasets, it may be desirable to aggregate only a few columns. For example, in the preceding dataset, to compute means for just the data2 column and get the result as a DataFrame, we could write:

In [33]:
# grouped DataFram
df.groupby(['studio_key', 'department_key'])[['data2']].mean()

data2
studio_key department_key       
DC         Advertising         5
           Production         -6
Marvel     Advertising         4
           Production          5

In [34]:
# grouped Series
df.groupby(['studio_key', 'department_key'])['data2'].mean()

studio_key  department_key
DC          Advertising       5
            Production       -6
Marvel      Advertising       4
            Production        5
Name: data2, dtype: int64

### Grouping with Dictionaries and Series

To see different ways to work with grouping, let's create a DataFrame of student grades that have taken two courses.

In [35]:
# create another sample DataFrame
students = pd.DataFrame(np.random.randint(80,100,(5,5)),
                       columns=[1,2,3,4,5],
                       index=['Joe','Steve','Beth','Jim','Sue'])
# add some NA values
students.iloc[2:3,[1,2]] = np.nan 

students

,1,2,3,4,5
Joe,85,88.0,93.0,91,96
Steve,80,90.0,96.0,97,89
Beth,81,NaN,NaN,80,90
Jim,88,86.0,96.0,84,89
Sue,83,98.0,97.0,96,95


Let's say that we want to map course names to the specific quiz. We can do this with a mapping and then groupby this dictionary mapping.

In [36]:
# create mapping
mapping = {1:670,2:670,3:520,4:520,5:670,6:680}

In [37]:
# passing the mapping to the groupby object
by_column = students.groupby(mapping, axis=1)

# summing by the grouped mapping, notice the unused mapping is OK
by_column.mean()

,520,670
Joe,92.0,89.666667
Steve,96.5,86.333333
Beth,80.0,85.500000
Jim,90.0,87.666667
Sue,96.5,92.000000


We can also work with Series:

In [38]:
# create Series of
map_series = pd.Series(mapping)
map_series

1    670
2    670
3    520
4    520
5    670
6    680
dtype: int64

In [39]:
# pass Series to groupby object
students.groupby(map_series, axis=1).count()

,520,670
Joe,2,3
Steve,2,3
Beth,1,2
Jim,2,3
Sue,2,3


### Grouping with Functions

Any function passed as a group key will be called once per index value.

As an example, let's say we wanted to group students based on how many letters were in their name and find their median score. (Why we would ever want to do this? Who knows. Just go along with me here.)

In [40]:
# group by index length (in this case student name)
students.groupby(len).median()

,1,2,3,4,5
3,85,88.0,96.0,91,95
4,81,NaN,NaN,80,90
5,80,90.0,96.0,97,89


In [41]:
# let's rename the columns
students = students.rename(columns=mapping)
students

,670,670,520,520,670
Joe,85,88.0,93.0,91,96
Steve,80,90.0,96.0,97,89
Beth,81,NaN,NaN,80,90
Jim,88,86.0,96.0,84,89
Sue,83,98.0,97.0,96,95


In [42]:
# create a second key list
key_list = ['MA','NY','NY','MA','NY']

# groupby function, then by key_list
students.groupby([len, key_list]).mean()# len= number of letter in their names

670   670   520   520   670
3 MA  86.5  87.0  94.5  87.5  92.5
  NY  83.0  98.0  97.0  96.0  95.0
4 NY  81.0   NaN   NaN  80.0  90.0
5 NY  80.0  90.0  96.0  97.0  89.0

### Grouping by Index Levels

You can easily aggregate using one of the levels of a multi-index. Let's add a `gender` column to our `students` data and create a multi-index DataFrame.

In [43]:
# adding gender column
students['gender'] = ['M','M','F','M','F']
students

,670,670,520,520,670,gender
Joe,85,88.0,93.0,91,96,M
Steve,80,90.0,96.0,97,89,M
Beth,81,NaN,NaN,80,90,F
Jim,88,86.0,96.0,84,89,M
Sue,83,98.0,97.0,96,95,F


In [44]:
# creating multi-index
students = students.reset_index().set_index(['index','gender'])
students

,,670,670,520,520,670
index,gender,,,,,
Joe,M,85,88.0,93.0,91,96
Steve,M,80,90.0,96.0,97,89
Beth,F,81,NaN,NaN,80,90
Jim,M,88,86.0,96.0,84,89
Sue,F,83,98.0,97.0,96,95


In [45]:
# grouping by 'gender' index and counting number of quizzes taken by gender
students.groupby(level='gender').count()

,670,670,520,520,670
gender,,,,,
F,2,1,1,2,2
M,3,3,3,3,3


In [46]:
# let's remind oursleves of our DataFrame
flags.head()

,name,landmass,zone,area,population,language,religion,num_bars,num_stripes,num_colors,red,green,blue,gold,white,black,orange,mainhue,num_circles,num_crosses,num_saltires,num_quarters,num_sunstars,crescent,triangle,icon,animate,text,topleft_color,botright_color
0,Afghanistan,5,1,648,16,10,2,0,3,5,1,1,0,1,1,1,0,green,0,0,0,0,1,0,0,1,0,0,black,green
1,Albania,3,1,29,3,6,6,0,0,3,1,0,0,1,0,1,0,red,0,0,0,0,1,0,0,0,1,0,red,red
2,Algeria,4,1,2388,20,8,2,2,0,3,1,1,0,0,1,0,0,green,0,0,0,0,1,1,0,0,0,0,green,white
3,American-Samoa,6,3,0,0,1,1,0,0,5,1,0,1,1,1,0,1,blue,0,0,0,0,0,0,1,1,1,0,blue,red
4,Andorra,3,1,0,0,6,0,3,0,3,1,0,1,1,0,0,0,gold,0,0,0,0,0,0,0,0,0,0,blue,red


What is the sum of the `num_crosses` grouped by `religion`?

*Note: religion: 0=Catholic, 1=Other Christian, 2=Muslim, 3=Buddhist, 4=Hindu, 5=Ethnic, 6=Marxist, 7=Others*

In [47]:
# sum of the num_crosses grouped by religion
flags.groupby('religion')['num_crosses'].sum()

religion
0     2
1    26
2     0
3     1
4     0
5     0
6     0
7     0
Name: num_crosses, dtype: int64

What is the sum of `crescent` grouped by `religion`?

In [48]:
# sum of crescent grouped by religion
flags.groupby('religion')['crescent'].sum()

religion
0    0
1    0
2    8
3    1
4    1
5    0
6    1
7    0
Name: crescent, dtype: int64

What are the total value counts of all the colors in `mainhue` grouped by `religion`?

In [49]:
# total value counts of all the colors in mainhue grouped by religion
flags.groupby('religion')['mainhue'].value_counts()

religion  mainhue
0         red        15
          blue        9
          gold        6
          white       6
          green       3
          black       1
1         blue       24
          red        16
          white       9
          green       6
          gold        3
          black       1
          orange      1
2         red        15
          green      12
          gold        3
          black       2
          blue        2
          brown       1
          orange      1
3         red         4
          blue        1
          gold        1
          orange      1
          white       1
4         brown       1
          green       1
          orange      1
          red         1
5         red        10
          green       8
          gold        5
          blue        2
          black       1
          white       1
6         red        10
          blue        2
          white       2
          gold        1
7         white       3
          green       

What is the maximum `area` for each group that is grouped by `zone` and then `religion`? What is the minimum area?

In [50]:
# maximum area for each group that is grouped by zone and then religion
flags.groupby(['zone','religion'])['area'].max()

zone  religion
1     0             547
      1            1222
      2            2506
      3             678
      4            3268
      5            1284
      6           22402
      7             372
2     1            7690
      2            1904
      4               2
      5            1247
3     0            8512
      1              12
4     0            1973
      1            9976
      2            1240
      4             215
      5             323
      6             115
Name: area, dtype: int64

In [51]:
# minimum area for each group that is grouped by zone and then religion
flags.groupby(['zone','religion'])['area'].min()

zone  religion
1     0             0
      1             0
      2             0
      3             1
      4           140
      5            28
      6            29
      7            21
2     1             0
      2             2
      4             2
      5            26
3     0             4
      1             0
4     0             4
      1             0
      2           196
      4           215
      5            10
      6           115
Name: area, dtype: int64

Let's try to determine if there are more colors or shapes on the country flags.
1. Create a subset of the `flags` data and call it `flags_subset`. This subset should include the following attributes: 'red', 'green', 'blue', 'gold', 'white', 'black', 'orange', 'num_circles', 'num_crosses', 'num_saltires', 'num_sunstars', 'crescent',  and 'triangle'
2. Create a dictionary that maps all colors to the string `color` and all shapes to the string `shape`
3. Use the mapping dictionary from step 2 to calculate the sum of the colors and shapes for each instance.
4. *Bonus:* Sum up all the colors and shapes for all instances to determine if there are more colors or shapes on all the flags.

In [52]:
# create a subset of the data
flags_subset = flags[['red', 'green', 'blue', 'gold', 'white', 'black', 'orange', 'num_circles', 'num_crosses', 'num_saltires',
                'num_sunstars', 'crescent', 'triangle']]

In [53]:
#flags_subset

In [54]:
# create a dictionary that maps colors and shapes
mapping = { 'red':'color', 'green':'color', 'blue': 'color', 'gold': 'color', 'white':'color', 'black':'color',
           'orange':'color', 'num_circles': 'shape', 'num_crosses': 'shape', 'num_saltires': 'shape',
           'num_sunstars': 'shape', 'crescent': 'shape', 'triangle': 'shape'}

In [55]:
# groupby mapping and calculate sum
by_column = flags_subset.groupby(mapping, axis=1)
by_column.sum()

,color,shape
0,5,1
1,3,1
2,3,2
3,5,1
4,3,0
...,...,...
189,3,5
190,4,1
191,4,1
192,4,0


In [56]:
# calculate the sum of all instances
by_column.sum().sum()

color    658
shape    387
dtype: int64

### Data Aggregation

You can use aggregations of your own devising and additionally call any method that is also defined on the grouped object. 

Let's look at another simple example. First, let's create a similar DataFrame to our `students` that represents the grades of five quizzes these students received in one class.

In [57]:
# create another simple DataFrame
quiz_df = pd.DataFrame(np.random.randint(70,100,(8,5)),
                       columns=[1,2,3,4,5],
                       index=['Joe','Steve','Beth','Jim','Sue','James','Amy','Monika'])

# add a gender column for grouping
quiz_df['gender'] = ['M','M','F','M','F','M','F','F']
quiz_df

,1,2,3,4,5,gender
Joe,93,96,86,97,70,M
Steve,74,81,85,87,72,M
Beth,80,71,88,90,77,F
Jim,74,90,82,87,79,M
Sue,99,83,75,84,95,F
James,86,71,89,72,83,M
Amy,87,99,95,92,85,F
Monika,80,71,87,71,70,F


In [58]:
# groupby gender
grouped = quiz_df.groupby('gender')

In [59]:
grouped

In [60]:
# use agg and pass 'mean'
# notice you pass this as a string
grouped.agg('mean')

# notice that this is the same as passing the following:
#grouped.mean()

,1,2,3,4,5
gender,,,,,
F,86.50,81.0,86.25,84.25,81.75
M,81.75,84.5,85.50,85.75,76.00


In [61]:
grouped.mean() # same result with the above

,1,2,3,4,5
gender,,,,,
F,86.50,81.0,86.25,84.25,81.75
M,81.75,84.5,85.50,85.75,76.00


Now comes the fun part. Let's say that you wanted to know the range of the top score and bottom score for each quiz broken down by gender. We can create our own custom function to do this.

In [62]:
# create a simple custom function
def range_scores(arr):
    return arr.max() - arr.min()

# pass function to agg
grouped.agg(range_scores)

,1,2,3,4,5
gender,,,,,
F,19,28,20,21,25
M,19,25,7,25,13


Note that you can also pass the describe method to a grouped object.

In [63]:
grouped[1].describe()

,count,mean,std,min,25%,50%,75%,max
gender,,,,,,,,
F,4.0,86.50,8.962886,80.0,80.0,83.5,90.00,99.0
M,4.0,81.75,9.394147,74.0,74.0,80.0,87.75,93.0


In [64]:
grouped[2].describe() # column 2

,count,mean,std,min,25%,50%,75%,max
gender,,,,,,,,
F,4.0,81.0,13.266499,71.0,71.0,77.0,87.0,99.0
M,4.0,84.5,10.908712,71.0,78.5,85.5,91.5,96.0


Let's add another column that lists the student's level.

In [65]:
quiz_df['level'] = ['Senior','Junior','Senior','Senior','Junior','Junior','Senior','Junior']
quiz_df

,1,2,3,4,5,gender,level
Joe,93,96,86,97,70,M,Senior
Steve,74,81,85,87,72,M,Junior
Beth,80,71,88,90,77,F,Senior
Jim,74,90,82,87,79,M,Senior
Sue,99,83,75,84,95,F,Junior
James,86,71,89,72,83,M,Junior
Amy,87,99,95,92,85,F,Senior
Monika,80,71,87,71,70,F,Junior


Next, let's add an average quiz score for each student.

In [66]:
quiz_df['avg'] = quiz_df[[1,2,3,4,5]].mean(axis=1)
quiz_df

,1,2,3,4,5,gender,level,avg
Joe,93,96,86,97,70,M,Senior,88.4
Steve,74,81,85,87,72,M,Junior,79.8
Beth,80,71,88,90,77,F,Senior,81.2
Jim,74,90,82,87,79,M,Senior,82.4
Sue,99,83,75,84,95,F,Junior,87.2
James,86,71,89,72,83,M,Junior,80.2
Amy,87,99,95,92,85,F,Senior,91.6
Monika,80,71,87,71,70,F,Junior,75.8


Now, let's group by student level and then by gender. We will select only the `avg` column and see what the mean score is for the respective groupings.

In [67]:
# groupby level and gender
grouped = quiz_df.groupby(['level','gender'])

# selecting only the avg column
grouped_avg = grouped['avg']

# aggregating the mean
grouped_avg.agg('mean')


level   gender
Junior  F         81.5
        M         80.0
Senior  F         86.4
        M         85.4
Name: avg, dtype: float64

In [68]:
# same as above
grouped['avg'].mean()

level   gender
Junior  F         81.5
        M         80.0
Senior  F         86.4
        M         85.4
Name: avg, dtype: float64

If you pass a list of functions or function names instead, you get back a DataFrame with column names taken from the functions:

In [69]:
grouped_avg.agg(['mean','std',range_scores])

mean       std  range_scores
level  gender                              
Junior F       81.5  8.061017          11.4
       M       80.0  0.282843           0.4
Senior F       86.4  7.353911          10.4
       M       85.4  4.242641           6.0

You can also create a list of functions and pass this list to `agg`

In [70]:
# list of functions
functions = ['mean','std',range_scores]

grouped_avg.agg(functions)

mean       std  range_scores
level  gender                              
Junior F       81.5  8.061017          11.4
       M       80.0  0.282843           0.4
Senior F       86.4  7.353911          10.4
       M       85.4  4.242641           6.0

You can also change the name of the column when you aggregate like this:

In [71]:
grouped_avg.agg([('Average', 'mean'), ('Std Dev', 'std'), ('Range', range_scores)])
#its tuple, (name of column, function want to use)
# we can pass a list of multiple functions to the 'agg' method

Average   Std Dev  Range
level  gender                          
Junior F          81.5  8.061017   11.4
       M          80.0  0.282843    0.4
Senior F          86.4  7.353911   10.4
       M          85.4  4.242641    6.0

Finally, let's say that we want to apply a different function to separate columns of the DataFrame. You can pass a dictionary like this:

In [72]:
grouped.agg({1:'mean', 2:'median', 3:'count'})

1   2  3
level  gender             
Junior F       89.5  77  2
       M       80.0  76  2
Senior F       83.5  85  2
       M       83.5  93  2

### Apply (split-apply-combine)

The most general-purpose GroupBy method is `apply`. `apply` splits the object being manipulated into pieces, invokes the passed function on each piece, and then attempts to concatenate the pieces together.

Let's start by creating a new sample DataFrame of flights from Vienna to Charlotte.

In [73]:
# create sample flights data
flights = pd.DataFrame({
    'airline': ['Delta','Delta','Delta','Delta','Delta','United','United','United','United','Lufthansa','Lufthansa',
                'Lufthansa','Lufthansa','Lufthansa','Lufthansa','British Airways','British Airways','British Airways'],
    'price': np.random.randint(600,1000,18),
    'time': np.random.randint(8,16,18)
})

flights

,airline,price,time
0,Delta,933,15
1,Delta,984,8
2,Delta,617,12
3,Delta,841,8
4,Delta,805,10
5,United,983,8
6,United,850,8
7,United,731,9
8,United,934,14
9,Lufthansa,756,15


Now, let's create a custom function that selects `n` rows with the lowest values in a particular column.

In [74]:
# create custom function
def best(df, n=3, column='price'):# n=3 is top three rows(lowest price first)
    return df.sort_values(by=column)[:n]

In [75]:
# test on full data
best(flights)# by using that function, united airline has the lowest price

,airline,price,time
2,Delta,617,12
11,Lufthansa,649,12
13,Lufthansa,685,14


Now, let's group by `airlines` and call `apply` with this function.

In [76]:
flights.groupby('airline').apply(best)# this will return top 3 cheapest price airline 

airline  price  time
airline                                         
British Airways 17  British Airways    740     8
                15  British Airways    867     8
                16  British Airways    961    13
Delta           2             Delta    617    12
                4             Delta    805    10
                3             Delta    841     8
Lufthansa       11        Lufthansa    649    12
                13        Lufthansa    685    14
                10        Lufthansa    686     9
United          7            United    731     9
                6            United    850     8
                8            United    934    14

You can also add other arguments in the `apply` method. What if we wanted only the two shortest flights by airline?

In [77]:
# two shortest flights grouped by airline
flights.groupby('airline').apply(best, n=2, column='time')# n=2 is two shortest time for each airline

airline  price  time
airline                                         
British Airways 15  British Airways    867     8
                17  British Airways    740     8
Delta           1             Delta    984     8
                3             Delta    841     8
Lufthansa       12        Lufthansa    908     8
                10        Lufthansa    686     9
United          5            United    983     8
                6            United    850     8

In [78]:
flags.head()

,name,landmass,zone,area,population,language,religion,num_bars,num_stripes,num_colors,red,green,blue,gold,white,black,orange,mainhue,num_circles,num_crosses,num_saltires,num_quarters,num_sunstars,crescent,triangle,icon,animate,text,topleft_color,botright_color
0,Afghanistan,5,1,648,16,10,2,0,3,5,1,1,0,1,1,1,0,green,0,0,0,0,1,0,0,1,0,0,black,green
1,Albania,3,1,29,3,6,6,0,0,3,1,0,0,1,0,1,0,red,0,0,0,0,1,0,0,0,1,0,red,red
2,Algeria,4,1,2388,20,8,2,2,0,3,1,1,0,0,1,0,0,green,0,0,0,0,1,1,0,0,0,0,green,white
3,American-Samoa,6,3,0,0,1,1,0,0,5,1,0,1,1,1,0,1,blue,0,0,0,0,0,0,1,1,1,0,blue,red
4,Andorra,3,1,0,0,6,0,3,0,3,1,0,1,1,0,0,0,gold,0,0,0,0,0,0,0,0,0,0,blue,red


Group the data by `zone` and determine the difference between the zone's largest area and its smallest area. Do this by creating a custom function and passing it to the `agg` method.

In [79]:
grouped= flags.groupby(['zone'])[['area']]#.max()
grouped

In [80]:
### ENTER CODE HERE ###
def range_zonearea(df):
    return df.max()-df.min()

grouped.agg(range_zonearea)


,area
zone,
1,22402
2,7690
3,8512
4,9976


In [81]:
# lecturer method:
# grouop by zone
grouped = flags.groupby('zone')

# select only the area
grouped_zone_area = grouped['area']

# create custom function to return range
def col_range(arr):
    return arr.max()-arr.min()

# pass custom function to agg
grouped_zone_area.agg(col_range)


zone
1    22402
2     7690
3     8512
4     9976
Name: area, dtype: int64

Add a new column called `pop_den` to the DataFrame that represents the respective country's population density. Population density is defined as the population divided by the area.

In [82]:
#add a new column called pop_den
flags['pop_den']= flags['population']/flags['area']
flags.head()#['pop_den'] population=0 actually is million here.

,name,landmass,zone,area,population,language,religion,num_bars,num_stripes,num_colors,red,green,blue,gold,white,black,orange,mainhue,num_circles,num_crosses,num_saltires,num_quarters,num_sunstars,crescent,triangle,icon,animate,text,topleft_color,botright_color,pop_den
0,Afghanistan,5,1,648,16,10,2,0,3,5,1,1,0,1,1,1,0,green,0,0,0,0,1,0,0,1,0,0,black,green,0.024691
1,Albania,3,1,29,3,6,6,0,0,3,1,0,0,1,0,1,0,red,0,0,0,0,1,0,0,0,1,0,red,red,0.103448
2,Algeria,4,1,2388,20,8,2,2,0,3,1,1,0,0,1,0,0,green,0,0,0,0,1,1,0,0,0,0,green,white,0.008375
3,American-Samoa,6,3,0,0,1,1,0,0,5,1,0,1,1,1,0,1,blue,0,0,0,0,0,0,1,1,1,0,blue,red,NaN
4,Andorra,3,1,0,0,6,0,3,0,3,1,0,1,1,0,0,0,gold,0,0,0,0,0,0,0,0,0,0,blue,red,NaN


Group by `landmass` and determine the mean, median, standard deviation and range for the population density column. Call the columns 'Avg', 'Median', 'Std Dev' and 'Range' respectively.

In [83]:
#groupby landmass, select population density and aggregate the respective fuctions
grouped = flags.groupby(['landmass'])
grouped_pd= grouped['pop_den']
grouped_pd.agg([('AVG','mean'), ('Median', 'median'),('Std Dev', 'std'), ('Range', col_range)])


,AVG,Median,Std Dev,Range
landmass,,,,
1,0.088425,0.039121,0.101691,0.333333
2,0.022907,0.010895,0.046475,0.200000
3,0.111191,0.101084,0.088912,0.341463
4,0.038651,0.021672,0.076328,0.500000
5,0.318026,0.052838,0.921349,5.000000
6,0.026159,0.000975,0.049887,0.160000


Grouping the data by landmass, what is the max number of bars for each group, the average number of stripes, and the median value for number of colors?

In [84]:
#grouped by different functions
grouped.agg({'num_bars':'max','num_stripes':'mean', 'num_colors':'median'})

,num_bars,num_stripes,num_colors
landmass,,,
1,5,1.451613,3
2,3,2.176471,3
3,3,1.285714,3
4,3,1.942308,3
5,3,1.666667,3
6,0,0.400000,4


Now,
1. Create a custom function called `top` that returns the top 2 rows with the **largest** values in the `pop_den` column. 
2. Make sure that you do not include any rows with NaNs in the `pop_den` column.
3. Setup your function arguments so that you can change the number of rows to show and which column to sort by.
4. Group the `flags` data by `landmass` and use the apply function with your custom function.

In [86]:
# create custom function
def top(df, n=2, column= 'pop_den'):# .n=2 is top two rows(lowest 'pop_den')
    return df.dropna(subset=['pop_den']).sort_values(by=column, ascending=False)[:n]# :n mean top 2
        

In [87]:
# groupby landmass and apply custom function
flags.groupby('landmass').apply(top)

name  landmass  zone  area  population  language  \
landmass                                                                    
1        140      Puerto-Rico         1     4     9           3         2   
         52       El-Salvador         1     4    21           5         2   
2        173  Trinidad-Tobago         2     4     5           1         1   
         50           Ecuador         2     3   284           8         2   
3        121      Netherlands         3     1    41          14         6   
         16           Belgium         3     1    31          10         6   
4        111        Mauritius         4     2     2           1         1   
         143           Rwanda         4     2    26           5        10   
5        78         Hong-Kong         5     1     1           5         7   
         150        Singapore         5     1     1           3         7   
6        137      Philippines         6     1   300          48        10   
         82         Indonesia         6     2  1904         157        10   

              religion  num_bars  num_stripes  num_colors  red  green  blue  \
landmass                                                                      
1        140         0         0            5           3    1      0     1   
         52          0         0            3           2    0      0     1   
2        173         1         0            0           3    1      0     0   
         50          0         0            3           3    1      0     1   
3        121         1         0            3           3    1      0     1   
         16          0         3            0           3    1      0     0   
4        111         4         0            4           4    1      1     1   
         143         5         3            0           4    1      1     0   
5        78          3         0            0           6    1      1     1   
         150         3         0            2           2    1      0     0   
6        137         0         0            0           4    1      0     1   
         82          2         0            2           2    1      0     0   

              gold  white  black  orange mainhue  num_circles  num_crosses  \
landmass                                                                     
1        140     0      1      0       0     red            0            0   
         52      0      1      0       0    blue            0            0   
2        173     0      1      1       0     red            0            0   
         50      1      0      0       0    gold            0            0   
3        121     0      1      0       0     red            0            0   
         16      1      0      1       0    gold            0            0   
4        111     1      0      0       0     red            0            0   
         143     1      0      1       0     red            0            0   
5        78      1      1      0       1    blue            1            1   
         150     0      1      0       0   white            0            0   
6        137     1      1      0       0    blue            0            0   
         82      0      1      0       0     red            0            0   

              num_saltires  num_quarters  num_sunstars  crescent  triangle  \
landmass                                                                     
1        140             0             0             1         0         1   
         52              0             0             0         0         0   
2        173             0             0             0         0         1   
         50              0             0             0         0         0   
3        121             0             0             0         0         0   
         16              0             0             0         0         0   
4        111             0             0             0         0         0   
         143             0             0    

Using the above custom function (`top`), return the top 3 rows with the highest `population` grouped by `zone`.

In [88]:
#groupby zone and apply your custom function
flags.groupby('zone').apply(top,n=3, column='population')

name  landmass  zone   area  population  language  religion  \
zone                                                                            
1    37          China         5     1   9561        1008         7         6   
     81          India         5     1   3268         684         6         4   
     184          USSR         5     1  22402         274         5         6   
2    82      Indonesia         6     2   1904         157        10         2   
     153  South-Africa         4     2   1221          29         6         1   
     191         Zaire         4     2    905          28        10         5   
3    23         Brazil         2     3   8512         119         6         0   
     8       Argentina         2     3   2777          28         2         0   
     9       Argentine         2     3   2777          28         2         0   
4    183           USA         1     4   9363         231         1         1   
     112        Mexico         1     4   1973          77         2         0   
     180            UK         3     4    245          56         1         1   

          num_bars  num_stripes  num_colors  red  green  blue  gold  white  \
zone                                                                         
1    37          0            0           2    1      0     0     1      0   
     81          0            3           4    0      1     1     0      1   
     184         0            0           2    1      0     0     1      0   
2    82          0            2           2    1      0     0     0      1   
     153         0            3           5    1      1     1     0      1   
     191         0            0           4    1      1     0     1      0   
3    23          0            0           4    0      1     1     1      1   
     8           0            3           2    0      0     1     0      1   
     9           0            3           3    0      0     1     1      1   
4    183         0           13           3    1      0     1     0      1   
     112         3            0           4    1      1     0     0      1   
     180         0            0           3    1      0     1     0      1   

          black  orange mainhue  num_circles  num_crosses  num_saltires  \
zone                                                                      
1    37       0       0     red            0            0             0   
     81       0       1  orange            1            0             0   
     184      0       0     red            0            0             0   
2    82       0       0     red            0            0             0   
     153      0       1  orange            0            1             1   
     191      0       1   green            1            0             0   
3    23       0       0   green            1            0             0   
     8        0       0    blue            0            0             0   
     9        0       0    blue            0            0             0   
4    183      0       0   white            0            0             0   
     112      0       1   green            0            0             0   
     180      0       0     red            0            1             1   

          num_quarters  num_sunstars  crescent  triangle  icon  animate  text  \
zone                                                                            
1    37              0             5         0         0     0        0     0   
     81              0             0         0         0     1        0     0   
     184             0             1         0         0     1        0     0   
2    82              0             0         0         0     0        0     0   
     153             0             0         0         0     0        0     0   
     191             0             0         0         0     1        1     0   
3    23              0            22         0         0     0        0     1   
     8    

1. Create a second custom function called `bottom` that returns the 2 rows with the smallest values in the `pop_den` column. Do not include any rows with a `pop_den` of `0`.
2. Group the `flags` data by `landmass` and use the apply function with your custom function.

In [89]:
# create second custom function
def bottom(df, n=2, column= 'pop_den'):# n=2 is top two rows(lowest 'pop_den')
    return df[df[column]!=0].sort_values(by=column)[:n]

In [90]:
flags.groupby('landmass').apply(bottom)

name  landmass  zone  area  population  language  \
landmass                                                                     
1        31             Canada         1     4  9976          24         1   
         124         Nicaragua         1     4   128           3         2   
2        75             Guyana         2     4   215           1         1   
         21            Bolivia         2     3  1099           6         2   
3        130            Norway         3     1   324           4         6   
         58            Finland         3     1   337           5         9   
4        22           Botswana         4     2   600           1        10   
         100             Libya         4     1  1760           3         8   
5        115          Mongolia         5     1  1566           2        10   
         146      Saudi-Arabia         5     1  2150           9         8   
6        10          Australia         6     2  7690          15         1   
         134  Papua-New-Guinea         6     2   463           3         1   

              religion  num_bars  num_stripes  num_colors  red  green  blue  \
landmass                                                                      
1        31          1         2            0           2    1      0     0   
         124         0         0            3           2    0      0     1   
2        75          4         0            0           5    1      1     0   
         21          0         0            3           3    1      1     0   
3        130         1         0            0           3    1      0     1   
         58          1         0            0           2    0      0     1   
4        22          5         0            5           3    0      0     1   
         100         2         0            0           1    0      1     0   
5        115         6         3            0           3    1      0     1   
         146         2         0            0           2    0      1     0   
6        10          1         0            0           3    1      0     1   
         134         5         0            0           4    1      0     0   

              gold  white  black  orange mainhue  num_circles  num_crosses  \
landmass                                                                     
1        31      0      1      0       0     red            0            0   
         124     0      1      0       0    blue            0            0   
2        75      1      1      1       0   green            0            0   
         21      1      0      0       0     red            0            0   
3        130     0      1      0       0     red            0            1   
         58      0      1      0       0   white            0            1   
4        22      0      1      1       0    blue            0            0   
         100     0      0      0       0   green            0            0   
5        115     1      0      0       0     red            2            0   
         146     0      1      0       0   green            0            0   
6        10      0      1      0       0    blue            0            1   
         134     1      1      1       0   black            0            0   

              num_saltires  num_quarters  num_sunstars  crescent  triangle  \
landmass                                                                     
1        31              0             0             0         0         0   
         124             0             0             0         0         0   
2        75              0             0             0         0         1   
         21              0             0             0         0         0   
3        130             0             0             0         0         0   
         58              0             0             0         0         0   
4        22              0             0             0         0         0   
         100             0     

### More Pivot Tables and Cross-Tabulation

A pivot table aggregates a table of data by one or more keys, arranging the data in a rectangle with some of the group keys along the rows and some along the columns. Pivot tables in Python with pandas are made possible through groupby combined with reshape operations utilizing hierarchical indexing.

To show this, let's add a new column to our `flights` data that shows if the flight is in the morning or afternoon/evening.

In [91]:
# setup list
part_of_day = ['AM','PM']

# use random choice to select AM/PM for each row
time_of_day = np.random.choice(part_of_day,18)
time_of_day

array(['PM', 'PM', 'PM', 'AM', 'PM', 'PM', 'AM', 'AM', 'AM', 'PM', 'AM',
       'PM', 'AM', 'AM', 'AM', 'AM', 'PM', 'PM'], dtype='<U2')

In [92]:
# add new column to data
flights['time_of_day'] = time_of_day
flights

,airline,price,time,time_of_day
0,Delta,933,15,PM
1,Delta,984,8,PM
2,Delta,617,12,PM
3,Delta,841,8,AM
4,Delta,805,10,PM
5,United,983,8,PM
6,United,850,8,AM
7,United,731,9,AM
8,United,934,14,AM
9,Lufthansa,756,15,PM


Suppose you wanted to compute a table of price and time averages arranged by airline and time of day.

In [93]:
# average of price/time by airline/time of day
flights.pivot_table(index=['airline','time_of_day'])

price       time
airline         time_of_day                       
British Airways AM           867.000000   8.000000
                PM           850.500000  10.500000
Delta           AM           841.000000   8.000000
                PM           834.750000  11.250000
Lufthansa       AM           747.500000  11.250000
                PM           702.500000  13.500000
United          AM           838.333333  10.333333
                PM           983.000000   8.000000

You can choose just a select column or group of columns.

In [94]:
# select only price
flights.pivot_table('price', index=['airline','time_of_day'])

price
airline         time_of_day            
British Airways AM           867.000000
                PM           850.500000
Delta           AM           841.000000
                PM           834.750000
Lufthansa       AM           747.500000
                PM           702.500000
United          AM           838.333333
                PM           983.000000

Or you can compute the average price and time broken down by time of day.

In [95]:
# average price/time broken down by time of day
flights.pivot_table(index=['airline'],columns='time_of_day')

price               time       
time_of_day              AM      PM         AM     PM
airline                                              
British Airways  867.000000  850.50   8.000000  10.50
Delta            841.000000  834.75   8.000000  11.25
Lufthansa        747.500000  702.50  11.250000  13.50
United           838.333333  983.00  10.333333   8.00

If you include `margins=True`, it will compute group statistics for all the data within a single tier.

In [96]:
# include margins=True
flights.pivot_table('price', index='airline', columns='time_of_day', margins=True)#  displat all column(averaege price for all airline) and all rows

time_of_day,AM,PM,All
airline,,,
British Airways,867.000000,850.500000,856.000000
Delta,841.000000,834.750000,836.000000
Lufthansa,747.500000,702.500000,732.500000
United,838.333333,983.000000,874.500000
All,801.444444,825.333333,813.388889


In [97]:
flights[flights['airline'] == 'British Airways']['price'].mean()
#flights[flights['time_of_day'] == 'AM']['price'].mean()# 801.4444
#flights['price'].mean()# 813.3888889

856.0

The default function for a pivot table is `mean` although you can change it with the `aggfunc` argument.

In [98]:
# using count
flights.pivot_table(['price'], index=['airline'], columns='time_of_day', margins=True, aggfunc='count')

price       
time_of_day        AM PM All
airline                     
British Airways     1  2   3
Delta               1  4   5
Lufthansa           4  2   6
United              3  1   4
All                 9  9  18

A cross-tabulation (or crosstab) is a special case of a pivot table that computes group frequencies.

In [99]:
# using cross tab
pd.crosstab(flights['airline'], flights['time_of_day'], margins=True)

time_of_day,AM,PM,All
airline,,,
British Airways,1,2,3
Delta,1,4,5
Lufthansa,4,2,6
United,3,1,4
All,9,9,18


Create a pivot table using the `flags` data. Group the rows by `landmass` and use `median` as the aggregation function.

In [100]:
# create pivot table 
flags.pivot_table(index=['landmass'], aggfunc='median')

,animate,area,black,blue,crescent,gold,green,icon,language,num_bars,num_circles,num_colors,num_crosses,num_quarters,num_saltires,num_stripes,num_sunstars,orange,pop_den,population,red,religion,text,triangle,white,zone
landmass,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0,9.0,0,1,0,1,0,0,1,0,0,3,0,0,0,0,0.0,0,0.039121,0,1,1,0,0,1,4
2,0,407.0,0,1,0,1,0,0,2,0,0,3,0,0,0,3,0.0,0,0.010895,6,1,0,0,0,1,3
3,0,92.0,0,1,0,0,0,0,6,0,0,3,0,0,0,0,0.0,0,0.101084,8,1,1,0,0,1,1
4,0,298.5,0,0,0,1,1,0,8,0,0,3,0,0,0,1,0.0,0,0.021672,5,1,5,0,0,1,2
5,0,185.0,0,0,0,0,0,0,8,0,0,3,0,0,0,0,1.0,0,0.052838,10,1,2,0,0,1,1
6,0,2.0,0,1,0,1,0,0,1,0,0,4,0,0,0,0,2.5,0,0.000975,0,1,1,0,0,1,2


Create a pivot table grouping by `religion` in the index and summing the `crescent`, `num_crosses`, and `num_saltires`  columns.

In [111]:
#pivot table
flags.pivot_table(['crescent','num_crosses','num_saltires'], index=['religion'],aggfunc='sum')

,crescent,num_crosses,num_saltires
religion,,,
0,0,2,0
1,0,26,16
2,8,0,0
3,1,1,1
4,1,0,0
5,0,0,1
6,1,0,0
7,0,0,0


Create a pivot table grouping by `religion` in the index and `zone` in the columns. Use the `sum` function and select the `crescent`, `num_crosses`, and `num_saltires` columns. Add a total for each row and for each column. 

In [112]:
#pivot table , column without []also can
flags.pivot_table(['crescent','num_crosses','num_saltires'], index=['religion'],columns= ['zone'], margins=True, aggfunc='sum')

crescent                    num_crosses                      \
zone            1    2    3    4 All           1    2    3     4 All   
religion                                                               
0             0.0  NaN  0.0  0.0   0         1.0  NaN  0.0   1.0   2   
1             0.0  0.0  0.0  0.0   0         6.0  7.0  4.0   9.0  26   
2             6.0  1.0  NaN  1.0   8         0.0  0.0  NaN   0.0   0   
3             1.0  NaN  NaN  NaN   1         1.0  NaN  NaN   NaN   1   
4             1.0  0.0  NaN  0.0   1         0.0  0.0  NaN   0.0   0   
5             0.0  0.0  NaN  0.0   0         0.0  0.0  NaN   0.0   0   
6             1.0  NaN  NaN  0.0   1         0.0  NaN  NaN   0.0   0   
7             0.0  NaN  NaN  NaN   0         0.0  NaN  NaN   NaN   0   
All           9.0  1.0  0.0  1.0  11         8.0  7.0  4.0  10.0  29   

         num_saltires                     
zone                1    2    3    4 All  
religion                                  
0                 0.0  NaN  0.0  0.0   0  
1                 0.0  5.0  4.0  7.0  16  
2                 0.0  0.0  NaN  0.0   0  
3                 1.0  NaN  NaN  NaN   1  
4                 0.0  0.0  NaN  0.0   0  
5                 0.0  1.0  NaN  0.0   1  
6                 0.0  NaN  NaN  0.0   0  
7                 0.0  NaN  NaN  NaN   0  
All               1.0  6.0  4.0  7.0  18

Notice that the above output should have a lot of NaNs in the rows. See the [pandas documentation](https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html) to see how to compute the same pivot table but fill all NaNs with a `0`.

In [107]:
##create pivot table and fill NAs
flags.pivot_table(['crescent','num_crosses','num_saltires'], index=['religion'],columns= 'zone', margins=True, fill_value=0, aggfunc='sum')

crescent              num_crosses               num_saltires        \
zone            1  2  3  4 All           1  2  3   4 All            1  2  3   
religion                                                                      
0               0  0  0  0   0           1  0  0   1   2            0  0  0   
1               0  0  0  0   0           6  7  4   9  26            0  5  4   
2               6  1  0  1   8           0  0  0   0   0            0  0  0   
3               1  0  0  0   1           1  0  0   0   1            1  0  0   
4               1  0  0  0   1           0  0  0   0   0            0  0  0   
5               0  0  0  0   0           0  0  0   0   0            0  1  0   
6               1  0  0  0   1           0  0  0   0   0            0  0  0   
7               0  0  0  0   0           0  0  0   0   0            0  0  0   
All             9  1  0  1  11           8  7  4  10  29            1  6  4   

                 
zone      4 All  
religion         
0         0   0  
1         7  16  
2         0   0  
3         0   1  
4         0   0  
5         0   1  
6         0   0  
7         0   0  
All       7  18

Using `crosstab`, compute the group frequencies for `landmass` vs `religion`.

In [113]:
#create crosstab group frequency
pd.crosstab(flags['landmass'], flags['religion'])

religion,0,1,2,3,4,5,6,7
landmass,,,,,,,,
1,9,21,0,0,0,0,1,0
2,13,3,0,0,1,0,0,0
3,14,13,0,0,0,0,8,0
4,2,7,15,0,1,26,0,1
5,0,0,20,8,2,0,6,3
6,2,16,1,0,0,1,0,0


## Building a Machine Learning Model

Determine if we can predict a country's main religion based mostly on its flag's details.

In [114]:
# standard imports
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer

# dropping name column as this provides no additional data
flags = flags.drop('name', axis=1)

# creating features and response
X = flags.drop('religion', axis=1)
y = flags[['religion']]

# splitting data into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [115]:
# create list of numeric columns
num_col = ['area','population','num_bars','num_stripes','num_colors','num_circles','num_crosses',
          'num_saltires','num_quarters','num_sunstars','pop_den'] 

X_train_num = X_train[num_col]

# create pipeline for numeric columns to impute and scale data
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),
    ('std_scaler', StandardScaler())
])

In [120]:
X_train_num 

,area,population,num_bars,num_stripes,num_colors,num_circles,num_crosses,num_saltires,num_quarters,num_sunstars,pop_den
5,1247,7,0,2,3,0,0,0,0,1,0.005613
135,407,3,0,3,6,1,0,0,0,1,0.007371
122,0,0,0,1,3,0,0,0,0,6,NaN
167,185,10,0,3,4,0,0,0,0,2,0.054054
85,70,3,3,0,3,0,0,0,0,0,0.042857
...,...,...,...,...,...,...,...,...,...,...,...
106,0,0,0,0,3,0,0,0,0,0,NaN
14,143,90,0,0,2,1,0,0,0,0,0.629371
92,181,6,0,0,2,0,0,0,0,0,0.033149
179,236,13,0,6,5,1,0,0,0,0,0.055085


In [116]:
# create list of numeric attributes
num_attribs = list(X_train_num)

# create list of attributes to be One-Hot-Encoded
OHE_attribs = ['landmass','zone','language','mainhue','topleft_color','botright_color']

# create full pipeline 
full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs),
    ('OHE', OneHotEncoder(), OHE_attribs),
    ], remainder='passthrough')

In [121]:
num_attribs

['area',
 'population',
 'num_bars',
 'num_stripes',
 'num_colors',
 'num_circles',
 'num_crosses',
 'num_saltires',
 'num_quarters',
 'num_sunstars',
 'pop_den']

In [117]:
# run training and testing data through pipeline
X_train_prepared = full_pipeline.fit_transform(X_train)
X_test_prepared = full_pipeline.transform(X_test)

In [118]:
from sklearn.ensemble import RandomForestClassifier

# just using mostly default values for random forest, no grid search
rf = RandomForestClassifier(n_estimators=1000, random_state=42)
rf.fit(X_train_prepared, np.array(y_train).ravel())

RandomForestClassifier(n_estimators=1000, random_state=42)

In [119]:
# RandomForestRegressor Generalization Errors
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

y_preds = rf.predict(X_test_prepared)
acc_score_forest = accuracy_score(y_test, y_preds)
prec_score_forest = precision_score(y_test, y_preds, average='micro')
recall_score_forest = recall_score(y_test, y_preds, average='micro')

model_name = type(rf).__name__

print(f'Model {model_name} | Accuracy: {acc_score_forest}')
print(f'Model {model_name} | Precision: {prec_score_forest}')
print(f'Model {model_name} | Recall: {recall_score_forest}')

Model RandomForestClassifier | Accuracy: 0.6923076923076923
Model RandomForestClassifier | Precision: 0.6923076923076923
Model RandomForestClassifier | Recall: 0.6923076923076923


The bottom line was that we are able to predict with 69% accuracy a country's main religion based mostly on the details of its flag. This is not bad considering what little data we have to work with. 
